1) Requirements Setup 

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import binomtest as binom_test

from datetime import datetime, timedelta
from pathlib import Path
from scipy.stats import t
import os
from scipy import stats


2) Getting the relevant data from the Dataset 

In [72]:
# getting the data from ELia's dataset 
# paths 
Root_dir = os.getcwd()
Data_dir = os.path.join(Root_dir, "data_extraction", "raw_df")

# load the CSV + computing the log returns
def load_with_log_returns(csv_filename,lag = 1):
	"""
	Load a CSV from the Data_dir and compute log returns and also the Log Prices  on the 'Close' column.
	This function is robust: it will handle CSVs that have either 'Date' or 'Datetime' (or both).
	It raises a clear error if neither date column or the 'Close' column is present.
	"""
	path = os.path.join(Data_dir, csv_filename)
	# Read first without forcing parse_dates so we can detect which date columns exist
	df = pd.read_csv(path)
	# Parse available date columns robustly
	for col in ['Date', 'Datetime']:
		if col in df.columns:
			df[col] = pd.to_datetime(df[col], errors='coerce')
	# Prefer 'Date' as the index, otherwise use 'Datetime' if present
	if 'Date' in df.columns and not df['Date'].isna().all():
		df.set_index('Date', inplace=True)
	elif 'Datetime' in df.columns and not df['Datetime'].isna().all():
		df.set_index('Datetime', inplace=True)
	else:
		raise ValueError(f"CSV {path} must contain a 'Date' or 'Datetime' column. Found: {list(df.columns)}")
	# Ensure 'Close' exists
	if 'Close' not in df.columns:
		raise ValueError(f"CSV {path} must contain a 'Close' column. Found: {list(df.columns)}")
	# Compute log returns safely (first entry will be NaN)
	df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(lag))
	
	df['Log_Prices'] = np.log(df['Close'])
	return df





nasdaq_daily_df = load_with_log_returns('nasdaq_daily_df.csv')
nasdaq_weekly_df = load_with_log_returns('nasdaq_weekly_df.csv')
nasdaq_monthly_df = load_with_log_returns('nasdaq_monthly_df.csv')
nasdaq_hourly_df = load_with_log_returns('nasdaq_hourly_df.csv')

# The DataFrame uses 'Date' (or 'Datetime') as the index, so select the 'Log_Returns' column directly.
nasdaq_daily_log_returns = nasdaq_daily_df['Log_Returns'].dropna()
nasdaq_weekly_log_returns = nasdaq_weekly_df['Log_Returns'].dropna()
nasdaq_monthly_log_returns = nasdaq_monthly_df['Log_Returns'].dropna()
nasdaq_hourly_df_log_returns = nasdaq_hourly_df['Log_Returns'].dropna()

nasdaq_hourly_df_log_prices = nasdaq_hourly_df['Log_Prices'].dropna()
nasdaq_daily_df_log_prices = nasdaq_daily_df['Log_Prices'].dropna()
nasdaq_weekly_df_log_prices = nasdaq_weekly_df['Log_Prices'].dropna()
nasdaq_monthly_df_log_prices = nasdaq_monthly_df['Log_Prices'].dropna()


3) Testing the Martingale Properties of the Log Return 

In [73]:

def test_mean_returns(returns):
    """
    Basically a variant of the T-test where we test if the mean return is equal to zero for each respective frequency.

    Returns:
        (mean_return, std_return, se_return, t_stat, p_value_super, p_value_sub, p_value_two)

    p_value_super: one-sided p-value for H1: mean < 0 (supermartingale)
    p_value_sub:   one-sided p-value for H1: mean > 0 (submartingale)
    p_value_two:   two-sided p-value for H1: mean != 0
    """
    # converts it all to a big ol numpy and drops the NaNs
    r = returns.values if hasattr(returns, "values") else np.array(returns)
    r = np.asarray(r)
    r = r[~np.isnan(r)]

    n = len(r)
    if n < 2:
        raise ValueError("Not enough data points to perform the test.")

    mean_return = np.mean(r)
    std_return = np.std(r, ddof=1)
    se_return = std_return / np.sqrt(n)

    # handle zero-variance case --> it just a likkle safety net 
    if se_return == 0:
        t_stat = 0.0
    else:
        t_stat = mean_return / se_return

    # one-sided p-values 
    p_value_super = t.cdf(t_stat, df=n-1)   
    p_value_sub = 1 - p_value_super        
    p_value_two = 2 * min(p_value_super, p_value_sub)  

    return mean_return, std_return, se_return, t_stat, p_value_super, p_value_sub, p_value_two

def print_mean_return_results(returns, label="Mean Return"):
    (mean, std, se, t_stat,
     p_super, p_sub, p_two) = test_mean_returns(returns)

    n = len(returns)

    print("\n==============================")
    print(f"TEST – {label}")
    print("==============================")
    print(f"Number of obs:                    {n}")
    print(f"Mean return:                      {mean:.6e}")
    print(f"Std dev:                          {std:.6e}")
    print(f"Std error of mean:                {se:.6e}")
    print(f"t-statistic:                      {t_stat:.3f}")
    print(f"\nTwo-sided p-value (H1: mean ≠ 0): {p_two:.4g}")
    print(f"One-sided p-value (H1: mean < 0 → supermartingale): {p_super:.4g}")
    print(f"One-sided p-value (H1: mean > 0 → submartingale):   {p_sub:.4g}")

    # Interpretation
    print("\nInterpretation:")
    if p_two < 0.05:
        if mean > 0:
            print("→ Significant *positive* drift: SUBMARTINGALE behaviour.")
        else:
            print("→ Significant *negative* drift: SUPERMARTINGALE behaviour.")
    else:
        print("→ Mean return not significantly different from zero: MARTINGALE-compatible.")

print_mean_return_results(nasdaq_daily_log_returns,   "NASDAQ – DAILY log returns")
print_mean_return_results(nasdaq_weekly_log_returns,  "NASDAQ – WEEKLY log returns")
print_mean_return_results(nasdaq_monthly_log_returns, "NASDAQ – MONTHLY log returns")
print_mean_return_results(nasdaq_hourly_df_log_returns, "NASDAQ – HOURLY log returns")




TEST – NASDAQ – DAILY log returns
Number of obs:                    3992
Mean return:                      5.747994e-04
Std dev:                          1.297311e-02
Std error of mean:                2.053283e-04
t-statistic:                      2.799

Two-sided p-value (H1: mean ≠ 0): 0.005144
One-sided p-value (H1: mean < 0 → supermartingale): 0.9974
One-sided p-value (H1: mean > 0 → submartingale):   0.002572

Interpretation:
→ Significant *positive* drift: SUBMARTINGALE behaviour.

TEST – NASDAQ – WEEKLY log returns
Number of obs:                    828
Mean return:                      2.775642e-03
Std dev:                          2.598318e-02
Std error of mean:                9.029778e-04
t-statistic:                      3.074

Two-sided p-value (H1: mean ≠ 0): 0.002182
One-sided p-value (H1: mean < 0 → supermartingale): 0.9989
One-sided p-value (H1: mean > 0 → submartingale):   0.001091

Interpretation:
→ Significant *positive* drift: SUBMARTINGALE behaviour.

TEST – NASDAQ

4.1 CW ratio for the log returns 

In [74]:
# Strong random Walk Hypothesis (RW1)
# Apply the Cowles and Jones (1937) test to check for independence of returns.
# The test compares the frequency of sequences (two successive returns with same sign)
# to the frequency of reversals (two successive returns with opposite signs).

def cowles_jones_test(returns, name=""):
    """
    Apply the Cowles and Jones (1937) test for independence of returns.

    Parameters:
        returns : array-like or pandas Series of returns
        name    : optional label for printing

    Returns:
        dict with keys: NS, NR, CJ, z_stat, p_value
    """
    # prepare data
    r = returns.dropna().values if hasattr(returns, "dropna") else np.asarray(returns)
    non_zero_mask = r != 0
    r = r[non_zero_mask]

    if len(r) < 2:
        raise ValueError("Not enough observations (after removing zeros/NaNs) to perform the Cowles and Jones test.")

    signs = np.sign(r)
    prod = signs[:-1] * signs[1:]

    NS = np.sum(prod > 0)
    NR = np.sum(prod < 0)
    N = NS + NR

    if NR == 0 or N == 0:
        raise ValueError("Not enough variability in returns to perform the Cowles and Jones test.")

    CJ_hat = NS / NR

    # Under RW1 with mu = 0: P(sequence) = P(reversal) = 0.5
    p_hat = NS / N
    p0 = 0.5
    se = np.sqrt(p0 * (1 - p0) / N)
    z_stat = (p_hat - p0) / se

    # use normal distribution for z-test
    from scipy.stats import norm
    p_value = 2 * (1 - norm.cdf(abs(z_stat)))

    print(f"\n===== Cowles & Jones Test ({name}) =====")
    print(f"N_S (sequences, same sign) : {NS}")
    print(f"N_R (reversals, opp. sign) : {NR}")
    print(f"Total pairs N              : {N}")
    print(f"CĴ = N_S / N_R            : {CJ_hat:.4f}")
    print(f"p̂ = N_S / (N_S+N_R)       : {p_hat:.4f}")
    print(f"z-stat (H0: p = 0.5)       : {z_stat:.4f}")
    print(f"p-value (two-sided)        : {p_value:.6f}")
    if p_value < 0.05:
        print("→ Reject RW1 (μ=0) at 5% level.")
    else:
        print("→ Cannot reject RW1 (μ=0) at 5% level.")

    return {
        "NS": int(NS),
        "NR": int(NR),
        "CJ": float(CJ_hat),
        "z_stat": float(z_stat),
        "p_value": float(p_value)
    }

cj_daily   = cowles_jones_test(nasdaq_daily_log_returns,        "Daily log returns")
cj_hourly  = cowles_jones_test(nasdaq_hourly_df_log_returns,    "Hourly log returns")
cj_weekly  = cowles_jones_test(nasdaq_weekly_log_returns,       "Weekly log returns")
cj_monthly = cowles_jones_test(nasdaq_monthly_log_returns,      "Monthly log returns")



===== Cowles & Jones Test (Daily log returns) =====
N_S (sequences, same sign) : 1994
N_R (reversals, opp. sign) : 1996
Total pairs N              : 3990
CĴ = N_S / N_R            : 0.9990
p̂ = N_S / (N_S+N_R)       : 0.4997
z-stat (H0: p = 0.5)       : -0.0317
p-value (two-sided)        : 0.974741
→ Cannot reject RW1 (μ=0) at 5% level.

===== Cowles & Jones Test (Hourly log returns) =====
N_S (sequences, same sign) : 763
N_R (reversals, opp. sign) : 764
Total pairs N              : 1527
CĴ = N_S / N_R            : 0.9987
p̂ = N_S / (N_S+N_R)       : 0.4997
z-stat (H0: p = 0.5)       : -0.0256
p-value (two-sided)        : 0.979584
→ Cannot reject RW1 (μ=0) at 5% level.

===== Cowles & Jones Test (Weekly log returns) =====
N_S (sequences, same sign) : 424
N_R (reversals, opp. sign) : 402
Total pairs N              : 826
CĴ = N_S / N_R            : 1.0547
p̂ = N_S / (N_S+N_R)       : 0.5133
z-stat (H0: p = 0.5)       : 0.7655
p-value (two-sided)        : 0.443987
→ Cannot reject RW1 

4.2 CW ratio for the log prices

In [75]:
# Run Cowles & Jones on price or log-price series (fixed implementation)
def cowles_jones_test_prices(series, name="", is_log=True):
    """
    Cowles & Jones test applied to a series of prices or log-prices.
    
    Parameters:
        series : pandas Series or array-like (price levels or log-prices)
        name   : optional label for printing
        is_log : if True, treat `series` as log-prices and compute returns as differences;
                 if False, treat `series` as price levels and compute percentage returns.
    
    Returns:
        dict with keys: NS, NR, CJ, z_stat, p_value
    """
    # ensure pandas is available in cell scope
    s = series.dropna() if hasattr(series, "dropna") else pd.Series(series).dropna()
    p = s.values
    # remove exact zeros (relevant for level prices)
    non_zero_mask = p != 0
    p = p[non_zero_mask]
    if len(p) < 2:
        raise ValueError("Not enough observations (after removing zeros/NaNs) to perform the Cowles and Jones test.")
    if is_log:
        # log-returns are differences of log-prices
        returns = np.diff(p)
    else:
        # percentage returns for price levels
        returns = np.diff(p) / p[:-1]
    signs = np.sign(returns)
    prod = signs[:-1] * signs[1:]
    NS = int(np.sum(prod > 0))
    NR = int(np.sum(prod < 0))
    N = int(NS + NR)
    if NR == 0 or N == 0:
        raise ValueError("Not enough variability in returns to perform the Cowles and Jones test.")
    CJ_hat = NS / NR
    p_hat = NS / N
    p0 = 0.5
    se = np.sqrt(p0 * (1 - p0) / N)
    z_stat = (p_hat - p0) / se
    from scipy.stats import norm
    p_value = 2 * (1 - norm.cdf(abs(z_stat)))
    print(f"\n===== Cowles & Jones Test on Series ({name}) =====")
    print(f"N_S (sequences, same sign) : {NS}")
    print(f"N_R (reversals, opp. sign) : {NR}")
    print(f"Total pairs N              : {N}")
    print(f"CĴ = N_S / N_R            : {CJ_hat:.4f}")
    print(f"p̂ = N_S / (N_S+N_R)       : {p_hat:.4f}")
    print(f"z-stat (H0: p = 0.5)       : {z_stat:.4f}")
    print(f"p-value (two-sided)        : {p_value:.6f}")
    if p_value < 0.05:
        print("→ Reject RW1 (μ=0) at 5% level.")
    else:
        print("→ Cannot reject RW1 (μ=0) at 5% level.")
    return {
        "NS": int(NS),
        "NR": int(NR),
        "CJ": float(CJ_hat),
        "z_stat": float(z_stat),
        "p_value": float(p_value)
    }

# The precomputed variables `nasdaq_*_df_log_prices` are Series of log-prices (they were created above),
# so pass them directly and set is_log=True to compute differences
cj_daily_log_prices   = cowles_jones_test_prices(nasdaq_daily_df_log_prices,   "Daily log prices",   is_log=True)
cj_hourly_log_prices  = cowles_jones_test_prices(nasdaq_hourly_df_log_prices,  "Hourly log prices",  is_log=True)
cj_weekly_log_prices  = cowles_jones_test_prices(nasdaq_weekly_df_log_prices,  "Weekly log prices",  is_log=True)
cj_monthly_log_prices = cowles_jones_test_prices(nasdaq_monthly_df_log_prices, "Monthly log prices", is_log=True)


===== Cowles & Jones Test on Series (Daily log prices) =====
N_S (sequences, same sign) : 1993
N_R (reversals, opp. sign) : 1996
Total pairs N              : 3989
CĴ = N_S / N_R            : 0.9985
p̂ = N_S / (N_S+N_R)       : 0.4996
z-stat (H0: p = 0.5)       : -0.0475
p-value (two-sided)        : 0.962115
→ Cannot reject RW1 (μ=0) at 5% level.

===== Cowles & Jones Test on Series (Hourly log prices) =====
N_S (sequences, same sign) : 763
N_R (reversals, opp. sign) : 764
Total pairs N              : 1527
CĴ = N_S / N_R            : 0.9987
p̂ = N_S / (N_S+N_R)       : 0.4997
z-stat (H0: p = 0.5)       : -0.0256
p-value (two-sided)        : 0.979584
→ Cannot reject RW1 (μ=0) at 5% level.

===== Cowles & Jones Test on Series (Weekly log prices) =====
N_S (sequences, same sign) : 424
N_R (reversals, opp. sign) : 401
Total pairs N              : 825
CĴ = N_S / N_R            : 1.0574
p̂ = N_S / (N_S+N_R)       : 0.5139
z-stat (H0: p = 0.5)       : 0.8008
p-value (two-sided)        : 0.

In [76]:
# Testing Drift in 1 period returns (each frequency)

def drift_ttest(returns, name = "", alternative = "two-sided"):

    r = returns.dropna().values if hasattr(returns, "dropna") else np.asarray(returns)
    r = np.asarray(r)
    r = r[~np.isnan(r)]

    n = len(r)
    if n < 2:
        raise ValueError("Not enough observations to perform t-test (need at least 2).")

    mean_r = r.mean()
    std_r = r.std(ddof=1)
    se_r = std_r / np.sqrt(n)

    # coz we ave tha some dif 0 n tha
    if se_r == 0:
        t_stat = 0.0
    else:
        t_stat = mean_r / se_r
    df = n - 1

    if alternative == "greater":
        # H1: mean > 0
        p_value = 1 - stats.t.cdf(t_stat, df=df)
    elif alternative == "less":
        # H1: mean < 0
        p_value = stats.t.cdf(t_stat, df=df)
    else:
        # two-sided
        # use survival function for numerical stability
        p_value = 2 * min(stats.t.cdf(t_stat, df=df), 1 - stats.t.cdf(t_stat, df=df))

    print(f"\n===== Drift t-test (1-period, {name}) =====")
    print(f"n          : {n}")
    print(f"mean(r)    : {mean_r:.6e}")
    print(f"std(r)     : {std_r:.6e}")
    print(f"t-stat     : {t_stat:.4f}")
    print(f"p-value    : {p_value:.6f} ({alternative})")
    if p_value < 0.05:
        print("→ Reject H0: evidence of drift.")
    else:
        print("→ Cannot reject H0: mean not significantly different from 0.")
    return {"mean": mean_r, "std": std_r, "t_stat": t_stat, "p_value": p_value}

# 1-period drift tests
drift_ttest(nasdaq_hourly_df['Log_Returns'],  "Hourly log returns",  alternative="greater")
drift_ttest(nasdaq_daily_df['Log_Returns'],   "Daily log returns",   alternative="greater")
drift_ttest(nasdaq_weekly_df['Log_Returns'],  "Weekly log returns",  alternative="greater")
drift_ttest(nasdaq_monthly_df['Log_Returns'], "Monthly log returns", alternative="greater")


===== Drift t-test (1-period, Hourly log returns) =====
n          : 1528
mean(r)    : 1.090797e-04
std(r)     : 5.680545e-03
t-stat     : 0.7506
p-value    : 0.226501 (greater)
→ Cannot reject H0: mean not significantly different from 0.

===== Drift t-test (1-period, Daily log returns) =====
n          : 3992
mean(r)    : 5.747994e-04
std(r)     : 1.297311e-02
t-stat     : 2.7994
p-value    : 0.002572 (greater)
→ Reject H0: evidence of drift.

===== Drift t-test (1-period, Weekly log returns) =====
n          : 828
mean(r)    : 2.775642e-03
std(r)     : 2.598318e-02
t-stat     : 3.0739
p-value    : 0.001091 (greater)
→ Reject H0: evidence of drift.

===== Drift t-test (1-period, Monthly log returns) =====
n          : 190
mean(r)    : 1.241308e-02
std(r)     : 4.943746e-02
t-stat     : 3.4610
p-value    : 0.000333 (greater)
→ Reject H0: evidence of drift.


{'mean': np.float64(0.012413083223296532),
 'std': np.float64(0.04943745639891111),
 't_stat': np.float64(3.4609900423081084),
 'p_value': np.float64(0.000332670743255159)}

(4.3) Campbell, Lo MacKinlay (1997) 

In [79]:

def long_horizon_returns(log_price_series, h):
    lp = np.asarray(log_price_series.dropna())

    n_blocks = len(lp) // h
    if n_blocks <= 1:
        raise ValueError(f"Not enough data for horizon h={h}.")

    lp_trunc = lp[:n_blocks * h]
    lp_start = lp_trunc[0::h]
    lp_end   = lp_trunc[h-1::h]
    r_h = lp_end - lp_start
    return r_h


# 2. Campbell long-horizon drift t-test
def long_horizon_drift_ttest(log_price_series, name="", horizons=(1,5,20,60)):
    print(f"\n===== Campbell Long-Horizon Drift Test ({name}) =====")
    for h in horizons:
        try:
            r_h = long_horizon_returns(log_price_series, h)
        except ValueError as e:
            print(f"h={h}: {e}")
            continue
        
        n = len(r_h)
        mean_r = r_h.mean()
        std_r = r_h.std(ddof=1)
        se_r = std_r / np.sqrt(n)
        t_stat = mean_r / se_r if se_r != 0 else 0.0
        df = n - 1
        p_value = 1 - stats.t.cdf(t_stat, df=df)  # one-sided (greater)

        print(f"\nh = {h} steps")
        print(f"  n_blocks : {n}")
        print(f"  mean R^{h} : {mean_r:.6e}")
        print(f"  std(R^{h}) : {std_r:.6e}")
        print(f"  t-stat : {t_stat:.4f}")
        print(f"  p-value (H0: mean=0, greater) : {p_value:.6f}")
        if p_value < 0.05:
            print("  → Reject H0: drift visible at this horizon.")
        else:
            print("  → Cannot reject H0 at this horizon.")


# 3. Make sure your data has a Log_Price column
nasdaq_hourly_df["Log_Price"]  = np.log(nasdaq_hourly_df["Close"])
nasdaq_daily_df["Log_Price"]   = np.log(nasdaq_daily_df["Close"])
nasdaq_weekly_df["Log_Price"]  = np.log(nasdaq_weekly_df["Close"])
nasdaq_monthly_df["Log_Price"] = np.log(nasdaq_monthly_df["Close"])


# 4. Run the Campbell-style long-horizon drift tests
long_horizon_drift_ttest(nasdaq_hourly_df["Log_Price"],  "Hourly log prices",  horizons=(1,5,20,60))
long_horizon_drift_ttest(nasdaq_daily_df["Log_Price"],   "Daily log prices",   horizons=(1,5,20,60))
long_horizon_drift_ttest(nasdaq_weekly_df["Log_Price"],  "Weekly log prices",  horizons=(1,5,20,60))
long_horizon_drift_ttest(nasdaq_monthly_df["Log_Price"], "Monthly log prices", horizons=(1,5,20,60))



===== Campbell Long-Horizon Drift Test (Hourly log prices) =====

h = 1 steps
  n_blocks : 1529
  mean R^1 : 0.000000e+00
  std(R^1) : 0.000000e+00
  t-stat : 0.0000
  p-value (H0: mean=0, greater) : 0.500000
  → Cannot reject H0 at this horizon.

h = 5 steps
  n_blocks : 305
  mean R^5 : 5.949643e-04
  std(R^5) : 1.095416e-02
  t-stat : 0.9486
  p-value (H0: mean=0, greater) : 0.171801
  → Cannot reject H0 at this horizon.

h = 20 steps
  n_blocks : 76
  mean R^20 : 2.691898e-03
  std(R^20) : 2.232551e-02
  t-stat : 1.0511
  p-value (H0: mean=0, greater) : 0.148283
  → Cannot reject H0 at this horizon.

h = 60 steps
  n_blocks : 25
  mean R^60 : 7.353976e-03
  std(R^60) : 3.185076e-02
  t-stat : 1.1544
  p-value (H0: mean=0, greater) : 0.129842
  → Cannot reject H0 at this horizon.

===== Campbell Long-Horizon Drift Test (Daily log prices) =====

h = 1 steps
  n_blocks : 3993
  mean R^1 : 0.000000e+00
  std(R^1) : 0.000000e+00
  t-stat : 0.0000
  p-value (H0: mean=0, greater) : 0.500

(5) RW2 - Second Hypothesis - "Random Walk with Drift & Uncorrelated Innovations" --> Trivially Strong Random Walk